In [1]:
#load phython 3.10

import numpy as np
import matplotlib.pyplot as plt
#import pandas as pd
from scipy.fft import fft, ifft
from scipy import signal
from scipy.fft import fftshift
import scipy
#from mpl_toolkits.mplot3d.axes3d import Axes3D

from numpy.fft import fftn, ifftn



from scipy import signal
import scipy
from scipy.interpolate import interp1d


from scipy.interpolate import interp1d

#import sys 
#import GW_processing as gp
#from pycbc.filter import sigma


import random

from tqdm import tqdm

plt.rc('font', size=18)
import multiprocessing

#from specs_function import *

from functools import partial


In [ ]:
#function to compute the spectrogram

def spectogram_plot(ts,fs=4096,nperseg=128,noverlap=64, nfft=896, plot=False,cmap='hot'):
    '''takes the time series as input
fs=samplimg rate nperseg=length of segment, nfft=padding
'''
    #896
    f, t, Sxx = signal.spectrogram(ts, fs=fs,nperseg=nperseg,noverlap=noverlap, nfft=nfft, scaling='spectrum') #returns f-array of freq, t-array of segment times, Sxx-spectrogram of ts

    if plot:
        plt.figure(figsize=(12,9))
        plt.pcolor(t,f, Sxx, cmap =cmap)
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [sec]')
        #plt.xlim(7.5,15)
        #plt.ylim(0,250)
        plt.colorbar()
        plt.show()
    return Sxx


def window_aniso(a,fw=0.3,tw=1/490):


    window = signal.tukey(a.shape[0],alpha=fw)
    
    window=np.stack([window] * a.shape[1], axis=1)
    
    a *= window

    window = signal.tukey(a.shape[1],alpha=tw)
    
    window=np.stack([window] * a.shape[0], axis=0)
    
    a *= window
    
    return a


def cross_corr2D (x,y,plot=False):
    '''takes spectogram 2d array as input'''
    fftx = fftn(x)
    ffty=fftn(y)
    ret = ifftn(fftx * np.conjugate(ffty))
    ret = fftshift(ret, axes=1)
    ret = fftshift(ret, axes=0)
    
    if plot:
        plt.figure(figsize=(15,12))
        plt.pcolor(ret.real,cmap='coolwarm')
        plt.ylim(175,275)
        #plt.xlim(xmin,xmax)
        plt.colorbar()
        

    return ret.real

In [2]:


data_path='scratch'


def load_data30k(split_n=10):
    
    '''load and split data into segements, default value is 10'''
    
    white3kL=np.load(data_path+'/white10kL_30k.npy') #signal
    white3kH=np.load(data_path+'/white10kH_30k.npy')

    

    white3kL_s=np.split(white3kL,split_n,axis=0)
    white3kH_s=np.split(white3kH,split_n,axis=0)



    del white3kL
    del white3kH

    
    return white3kL_s,white3kH_s


In [3]:
#cross correlation function for single processor

def corr_stack_wave(white3kL,white3kH,f_cut=18,window=True):
    
    '''cross correlation function for single processor'''
    
    #f_cut selects the maximum frequency for the analysis

    
    #####. stack INJECTIONS#####

    T=490 # time segment in seconds
    
    sN=len(white3kL)//T  #number of segments, 10 for 500k and 6 for 30k

    stack_list_wave=[None]*sN 
    #stack_list_noise=[None]*sN
    

    for i in tqdm(range(0,sN)): 
        j=i+1
        slice_array_event_L1=white3kL[i*4096*T:j*4096*T] 
        slice_array_event_H1=white3kH[i*4096*T:j*4096*T]
              
        
        
        
        
        slice_array_event_L1Sxx=spectogram_plot(slice_array_event_L1,nperseg=78,nfft=896,noverlap=64,plot=False)
        slice_array_event_H1Sxx=spectogram_plot(slice_array_event_H1,nperseg=78,nfft=896,noverlap=64,plot=False)
      
                
               
        
        slice_array_event_L1Sxx=slice_array_event_L1Sxx[:f_cut,:] #wave
        slice_array_event_H1Sxx=slice_array_event_H1Sxx[:f_cut,:]
        
        
        
        if window:
            
            slice_array_event_L1Sxx=window_aniso(slice_array_event_L1Sxx,fw=0.1)      
            slice_array_event_H1Sxx=window_aniso(slice_array_event_H1Sxx,fw=0.1)

        
        

        event_slice_2Dcross=cross_corr2D(slice_array_event_L1Sxx.real,slice_array_event_H1Sxx.real)
        


        
        stack_list_wave[i]=event_slice_2Dcross
        
        
        
        
        del slice_array_event_L1Sxx
        del slice_array_event_H1Sxx

        

    
    return stack_list_wave


#multiprocessing and returning the corr stack

def specs_analysis(fcut):
    
    '''multi processing cross correlation'''

    white3kL_s,white3kH_s=load_data30k() #load and split data(noise and waves)
    
    
    
    items=zip(white3kL_s,white3kH_s)
    if __name__ == "__main__":
    # input list
    
    # creating a pool object
        p = multiprocessing.Pool(10)

    # map list to target function
        result = p.starmap_async(partial(corr_stack_wave,f_cut=fcut,window=False),items ) #spectrogram correlation
        
        #result.wait()
        print("closing")
        p.close()
        print("Joining")
        p.join()
        
        
    result=result.get()    
    resultnp=np.array(result)
    
    sig=resultnp[:,:,:,:] # selecting signal

    
    del result
    del resultnp
    
    
    sigAll=sig.reshape(sig.shape[0]*sig.shape[1],sig.shape[2],sig.shape[3])
    



    del white3kL_s
    del white3kH_s

    
     
    return sigAll


In [4]:
# aperture formalism compute covariance etc


def snr_ap2(data,ep):    
    
    # stack_var_event=np.var(data,axis=0)
    
    
    stack_mean_event=np.mean(data,axis=0)

    
    t0=(stack_mean_event.shape[1])//2
    
    Nt=t0//ep//2-3 #20
    
    tl=t0-ep #10
    th=t0+ep
    
    del data

    
    corr_aperture_lst=[]

    for i in range(1,int(Nt)):
        j=i+1
        corr_aperture=np.mean(stack_mean_event[:,2*ep*i:2*ep*j]**1,axis=1,dtype="float64") #20
    
        beg=t0+(2*ep*(i+2)) #20
        end=t0+(2*ep*(j+2))   #+2 for avoiding the center 
    
        corr_aperture2=np.mean(stack_mean_event[:,beg:end]**1,axis=1,dtype="float64")
    
    
        corr_aperture_lst.append(corr_aperture)
    
        corr_aperture_lst.append(corr_aperture2)
    
    
    P_back=np.mean(corr_aperture_lst,axis=0,dtype="float64")
    
    P_back=np.array(P_back,dtype="float64")
    

    
    #background subbtraction
    
    stack_mean_subr=[]    
    for i in tqdm(range(stack_mean_event.shape[1])):
        diff=stack_mean_event[:,i]-P_back
        #stack_mean_sub=np.concatenate((stack_mean_sub,diff))
        stack_mean_subr.append(diff)
            
        
    stack_mean_subt=np.array(stack_mean_subr)

    stack_mean_sub=np.transpose(stack_mean_subt)
    
    del stack_mean_subr
    
    
    
    
    P_app=np.mean(stack_mean_sub[:,tl:th]**1,axis=1)
    
    P_clean=P_app
    
    # calculation of p_clean for non zero \Delta t
    
    P_clean_stack=[]
    
    for i in range(-int(Nt/10),int(Nt/10)):
       
    
        tli=(t0+i*2*ep)-ep #10
        thi=(t0+i*2*ep)+ep
        
        P_app_i=np.mean(stack_mean_event[:,tli:thi]**1,axis=1) #event or sub
    
        P_clean_i=P_app_i-P_back # if event then include back
        
        P_clean_stack.append(P_clean_i)
            
    
    # var
    
    var=np.var(corr_aperture_lst,axis=0) #var of estimator
         
    #cov
        
    cov2=np.cov(corr_aperture_lst,rowvar=False)
    
    del corr_aperture_lst
    
    

    return stack_mean_sub,P_clean,P_back,var,cov2,P_clean_stack
    
    

In [5]:

dataw=specs_analysis(fcut=14*8)

stack_mean_sub,P_clean,P_back,var,cov2,P_clean_stack=snr_ap2(dataw,6)


closing
Joining


100%|██████████| 143355/143355 [00:00<00:00, 413066.77it/s]


In [8]:
def snr(df,peak=False):
    
    if peak:
        
        f_llim=P_clean.shape[0]//2-1
        f_ulim=P_clean.shape[0]//2
        
    else:    
        f_llim=P_clean.shape[0]//2-df
        f_ulim=P_clean.shape[0]//2+df



    pinverseLH = np.linalg.pinv(cov2[f_llim:f_ulim,f_llim:f_ulim])
        
    snr_cov_p=np.dot(pinverseLH,P_clean[f_llim:f_ulim])
    snr_cov=np.dot(P_clean[f_llim:f_ulim],snr_cov_p)
        
    return np.sqrt(snr_cov)

In [9]:
snr(2,peak=True)

7.969113374973868

# correlation analysis

In [ ]:
# obtaining the correlation for parameter estimation(without noise)

def Time_freq_correlation(data,ep=3):
    
    ''' takes a stack of TF correlation as input i.e data
       ep is the size of the Delta t window
    '''
    
    
    
    TF_corr=np.mean(data,axis=0)
    
    
    t0=(TF_corr.shape[1])//2

    
    tl=t0-ep
    th=t0+ep
    
    deltaf_corr = np.mean(TF_corr[:,tl:th],axis=1)
    
    return TF_corr, deltaf_corr

In [ ]:
# obtaining the correlation for parameter estimation

# def Time_freq_correlation(data):
    
#     stack_mean_event=np.mean(data,axis=0)
    
#     return stack_mean_event

In [ ]:

dataw=specs_analysis(fcut=14*8)

TF_corr, deltaf_corr=Time_freq_correlation(dataw)